In [4]:
import numpy as np
import pandas as pd
import fuzzymatcher
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [27]:
df = pd.read_csv('title4.csv', encoding = "ISO-8859-1", dtype='unicode')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66802 entries, 0 to 66801
Data columns (total 24 columns):
id                 66802 non-null object
title              66802 non-null object
imdb_index         34 non-null object
kind_id            66802 non-null object
production_year    66802 non-null object
imdb_id            0 non-null object
phonetic_code      57286 non-null object
episode_of_id      61058 non-null object
season_nr          56174 non-null object
episode_nr         56144 non-null object
series_years       4741 non-null object
md5sum             66802 non-null object
id.1               66802 non-null object
movie_id           66802 non-null object
company_id         66802 non-null object
company_type_id    66802 non-null object
note               33586 non-null object
id.2               66802 non-null object
name               66802 non-null object
country_code       66802 non-null object
imdb_id.1          0 non-null object
name_pcode_nf      66802 non-null object
n

In [28]:
df_right = df[df.kind_id != "7"]

df_right.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5744 entries, 0 to 66801
Data columns (total 24 columns):
id                 5744 non-null object
title              5744 non-null object
imdb_index         34 non-null object
kind_id            5744 non-null object
production_year    5744 non-null object
imdb_id            0 non-null object
phonetic_code      5744 non-null object
episode_of_id      0 non-null object
season_nr          0 non-null object
episode_nr         0 non-null object
series_years       4741 non-null object
md5sum             5744 non-null object
id.1               5744 non-null object
movie_id           5744 non-null object
company_id         5744 non-null object
company_type_id    5744 non-null object
note               3672 non-null object
id.2               5744 non-null object
name               5744 non-null object
country_code       5744 non-null object
imdb_id.1          0 non-null object
name_pcode_nf      5744 non-null object
name_pcode_sf      5744 non-n

In [29]:
df_right.describe()

,id,title,imdb_index,kind_id,production_year,imdb_id,phonetic_code,episode_of_id,season_nr,episode_nr,...,company_id,company_type_id,note,id.2,name,country_code,imdb_id.1,name_pcode_nf,name_pcode_sf,md5sum.1
count,5744,5744,34,5744,5744,0.0,5744,0.0,0.0,0.0,...,5744,5744,3672,5744,5744,5744,0.0,5744,5744,5744
unique,2397,2326,3,2,20,0.0,1775,0.0,0.0,0.0,...,863,2,1067,863,863,1,0.0,708,702,863
top,2651604,The Jack Benny Program,I,2,1959,NaN,A3153,NaN,NaN,NaN,...,83,1,(TV),83,National Broadcasting Company (NBC),[us],NaN,N3541,N3541,9bb2ebf35b863c1467dee49d0b842952
freq,27,28,20,4741,562,NaN,144,NaN,NaN,NaN,...,941,3775,138,941,941,5744,NaN,941,941,941


In [30]:
df_left = pd.read_csv('enc.csv', encoding = "ISO-8859-1")
df_left.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1732 entries, 0 to 1731
Data columns (total 12 columns):
Number                 1732 non-null int64
Program Title          1732 non-null object
Program Type           1731 non-null object
Program Genre          1731 non-null object
Network                1704 non-null object
First Air Month        131 non-null float64
First Air Day          123 non-null float64
First Air Year         1732 non-null int64
Last Air Month         2 non-null float64
Last Air Day           2 non-null float64
Last Air Year          895 non-null object
Program Description    1732 non-null object
dtypes: float64(4), int64(2), object(6)
memory usage: 162.5+ KB


In [31]:
df_left["Program Type"].value_counts()

Series               1549
Pilot                 120
Unaired Pilot          31
Series                 10
Live Test Program       5
Test Film               3
Experimental            3
S eries                 2
Serial                  2
Pilot                   2
 {Series                1
Test Program            1
S enes                  1
{Smes                   1
Name: Program Type, dtype: int64

In [32]:
df_right = df_right.astype(str).apply(lambda x: x.str.lower())
df_left = df_left.astype(str).apply(lambda x: x.str.lower())

In [33]:
df_right.drop(df_right.columns[[2,5,7,8,9, 20]], axis=1, inplace=True)

In [34]:
df_right.head()

,id,title,kind_id,production_year,phonetic_code,series_years,md5sum,id.1,movie_id,company_id,company_type_id,note,id.2,name,country_code,name_pcode_nf,name_pcode_sf,md5sum.1
0,10924,1948 world series,2,1948,w6432,1948-????,369f2c5308bfdda965e8ac9f43fb6966,3076,10924,66,1,(1948) (usa) (tv),66,columbia broadcasting system (cbs),[us],c4516,c4516,7ae3ee174d2d4b06fc3d2ba6d7dc2fab
1,63168,abe burrows' almanac,2,1950,a1624,1950-????,73511f5cea12328331c9f3cfdca47334,15574,63168,66,1,nan,66,columbia broadcasting system (cbs),[us],c4516,c4516,7ae3ee174d2d4b06fc3d2ba6d7dc2fab
2,73174,action in the afternoon,2,1953,a2353,1953-????,7f69e95080d2d8e28d20f13d0ecf1f65,17407,73174,66,1,(1953) (usa) (tv) (original airing),66,columbia broadcasting system (cbs),[us],c4516,c4516,7ae3ee174d2d4b06fc3d2ba6d7dc2fab
3,73584,actor's studio,2,1948,a2362,1948-1950,09d66a0785678d200f33381e1a9db078,17507,73584,66,1,(1949-1950) (usa) (tv),66,columbia broadcasting system (cbs),[us],c4516,c4516,7ae3ee174d2d4b06fc3d2ba6d7dc2fab
34,78738,adventure showcase,2,1959,a3153,1959-????,cedfa222a39b180f357804d025fc0a8c,18489,78738,66,1,(1959) (usa) (tv),66,columbia broadcasting system (cbs),[us],c4516,c4516,7ae3ee174d2d4b06fc3d2ba6d7dc2fab


In [35]:
df_right.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5744 entries, 0 to 66801
Data columns (total 18 columns):
id                 5744 non-null object
title              5744 non-null object
kind_id            5744 non-null object
production_year    5744 non-null object
phonetic_code      5744 non-null object
series_years       5744 non-null object
md5sum             5744 non-null object
id.1               5744 non-null object
movie_id           5744 non-null object
company_id         5744 non-null object
company_type_id    5744 non-null object
note               5744 non-null object
id.2               5744 non-null object
name               5744 non-null object
country_code       5744 non-null object
name_pcode_nf      5744 non-null object
name_pcode_sf      5744 non-null object
md5sum.1           5744 non-null object
dtypes: object(18)
memory usage: 852.6+ KB


In [36]:
df_left.drop(df_left.head().columns[[5,6,8,9]], axis=1, inplace=True)

In [37]:
df_left.head()

,Number,Program Title,Program Type,Program Genre,Network,First Air Year,Last Air Year,Program Description
0,5137,mary kay and johnny.,series,comedy,"dumont, nbc, cbs",1947,1950,the marital misadventures of mary kay and joh...
1,7531,the small fry club.,series,children,dumont,1947,1951,"bob emery hosts a program of stories, game co..."
2,1738,crawford mystery theater.,series,game,dumont,1947,1952,the alternate title for public prosecutor (wh...
3,6636,public prosecutor.,series,mystery,"syn., dumont",1947,1952,the syndicated version of the series (1947-48...
4,4323,juvenile jury.,series,children,"nbc, cbs",1947,1955,the program features a panel of five children...


In [38]:
df_right["title"].head()

0           1948 world series
1        abe burrows' almanac
2     action in the afternoon
3              actor's studio
34         adventure showcase
Name: title, dtype: object

In [39]:
df_left["Program Title"] = df_left["Program Title"].str.replace('[^\w\s]','')

In [40]:
df_left["Program Title"].head()

0          mary kay and johnny 
1           the small fry club 
2     crawford mystery theater 
3            public prosecutor 
4                juvenile jury 
Name: Program Title, dtype: object

In [41]:
(process.extract(x, df_left, scorer= fuzz.ratio) for x in df_right)

<generator object <genexpr> at 0x10dc62780>

In [42]:
#This works but needs tweaking

names_array=[]
ratio_array=[]

def match_names(wrong_names,correct_names):
    for row in wrong_names:
        x=process.extractOne(row, correct_names)
        names_array.append(x[0])
        ratio_array.append(x[1])
    return names_array,ratio_array
 
 
#imdb names dataset

wrong_names=df_left["Program Title"].dropna().values
 
#enc names dataset
correct_names=df_right["title"].dropna().values
 
name_match,ratio_match=match_names(wrong_names,correct_names)
 
df_left['correct_name']=pd.Series(name_match)
df_left['name_ratio']=pd.Series(ratio_match)
 
 
print(df_left[['Program Title','correct_name','name_ratio']].head(10))

                    Program Title              correct_name  name_ratio
0            mary kay and johnny        mary kay and johnny         100
1             the small fry club             small fry club          95
2       crawford mystery theater   crawford mystery theatre          96
3              public prosecutor          public prosecutor         100
4                  juvenile jury              juvenile jury         100
5   the kraft television theater   kraft television theatre          88
6           kukla fran and ollie      kukla, fran and ollie          98
7                    howdy doody            hollywood today          62
8             midwestern hayride         midwestern hayride         100
9                  actors studio             actor's studio          96


In [18]:
#df_left.to_csv("testing.csv")

In [43]:
df_merge = pd.merge(df_left, df_right, left_on= "correct_name", right_on = "title")

In [44]:
print(df_merge.head(10))

  Number          Program Title Program Type Program Genre            Network  \
0   5137   mary kay and johnny        series        comedy   dumont, nbc, cbs   
1   5137   mary kay and johnny        series        comedy   dumont, nbc, cbs   
2   5137   mary kay and johnny        series        comedy   dumont, nbc, cbs   
3   5137   mary kay and johnny        series        comedy   dumont, nbc, cbs   
4   5137   mary kay and johnny        series        comedy   dumont, nbc, cbs   
5   5137   mary kay and johnny        series        comedy   dumont, nbc, cbs   
6   5137   mary kay and johnny        series        comedy   dumont, nbc, cbs   
7   6006                oh kay        series       variety                abc   
8   6006                oh kay        series       variety                abc   
9   6006                oh kay        series       variety                abc   

  First Air Year Last Air Year  \
0           1947          1950   
1           1947          1950   
2     

In [45]:
df_merge.to_csv("merged.csv")
###########

In [155]:
#DOES NOT WORK
#Columns to match on from df_left

df_left.dropna().values
df_right.dropna().values

left_on = ["Program Title", "First Air Year"]

# Columns to match on from df_right
right_on = ["title", "production_year"]

fuzzymatcher.link_table(df_left, df_right, left_on, right_on)

ZeroDivisionError: float division by zero

In [158]:
#DOES NOT WORK
fuzzymatcher.fuzzy_left_join(df_left, df_right, left_on = "Program Title", right_on = "title")

ZeroDivisionError: float division by zero